In [1]:
from binance.client import Client
from binance.enums import *
import time

# 🧩 Step 1: Connect to Binance Testnet
API_KEY = "eOUQOwsdCjiNMUDbNsrTV0TUPPF7rAiEXiWtk5CMEGClviPoqBqFhm1KFddUtyoE"
API_SECRET = "rpPI79M1aoh8sc7oeZJTj8Fox3ppcBPhtBvWXtTBCWW65q4Ys23tBxo5MZ8tgIZv"

client = Client(API_KEY, API_SECRET, testnet=True)  # testnet=True means fake money

# 🎯 Step 2: Choose which assets to rebalance
TARGETS = {
    "BTCUSDT": 0.5,  # 50% Bitcoin
    "ETHUSDT": 0.5   # 50% Ethereum
}

In [2]:
# 📈 Step 3: Helper - Get current prices
def get_prices(symbols):
    prices = {}
    for symbol in symbols:
        ticker = client.get_symbol_ticker(symbol=symbol)
        prices[symbol] = float(ticker['price'])
    return prices

# 💰 Step 4: Helper - Get current balances
def get_usdt_balances(prices):
    balances = {}
    for symbol in TARGETS.keys():
        coin = symbol.replace("USDT", "")
        amount = float(client.get_asset_balance(asset=coin)['free'])
        balances[symbol] = amount * prices[symbol]
    total_value = sum(balances.values())
    return balances, total_value

# 🤖 Step 5: Rebalance Logic
def rebalance():
    symbols = list(TARGETS.keys())
    prices = get_prices(symbols)
    balances, total_value = get_usdt_balances(prices)

    print("\n----- Portfolio Before Rebalance -----")
    for s in symbols:
        percent = balances[s] / total_value * 100
        print(f"{s}: ${balances[s]:.2f} ({percent:.2f}%)")

    for s in symbols:
        target_value = total_value * TARGETS[s]
        difference = balances[s] - target_value

        if abs(difference) > 5:  # ignore tiny changes
            if difference > 0:
                # sell some of this coin
                qty = round(difference / prices[s], 5)
                print(f"🔴 Selling {qty} of {s} worth ${difference:.2f}")
                # Uncomment below when you're ready:
                # client.order_market_sell(symbol=s, quantity=qty)
            else:
                # buy some of this coin
                qty = round(abs(difference) / prices[s], 5)
                print(f"🟢 Buying {qty} of {s} worth ${abs(difference):.2f}")
                # Uncomment below when you're ready:
                # client.order_market_buy(symbol=s, quantity=qty)

    print("✅ Rebalancing completed!\n")

In [3]:
# 🔁 Step 6: Run the bot continuously
while True:
    rebalance()
    time.sleep(3600)  # check every 1 hour

BinanceAPIException: APIError(code=-2015): Invalid API-key, IP, or permissions for action.